In [1]:
import os, sys
import json
import cv2
import numpy as np

def gen_funds_format():
    set_path = "./data/yanbao_v1/"
    img_set_path = set_path + "研报JPEG/"
    max_norm_size = 1000
    input_path = img_set_path
    file_paths = []
    if os.path.isfile(input_path):
        file_paths.append(input_path)
    else:
        for root, _, fs in os.walk(input_path):
            for f in fs:
                file_paths.append(os.path.join(root, f))
    print(len(file_paths), file_paths[:5])

    save_label_bbox_path = set_path + "yanbao_v1_box.txt"
    save_label_image_path = set_path + "yanbao_v1_image.txt"
    foutb = open(save_label_bbox_path, "wb")
    fouti = open(save_label_image_path, "wb")

    def get_norm_resize_bbox(box, img_height, img_width, max_norm_size=1000):
        im_size_max = max(img_height, img_width)
        im_scale = float(max_norm_size) / float(im_size_max)
        box = np.array(box) * im_scale
        box = [int(np.round(i)) for i in box]
        return box

    for fno in range(len(file_paths)):
        if fno % 100 == 0:
            print(fno, len(file_paths))
        img_path = file_paths[fno]
        if 'DS_Store' in img_path:
            continue
        img = cv2.imread(img_path)
        img_height, img_width = img.shape[0:2]
        ocr_res_path = img_path.replace('研报JPEG', 'result') + ".txt"
        with open(ocr_res_path, "rb") as fin:
            lines = fin.readlines()
            bbox_num = len(lines)
            for bno in range(bbox_num):
                line = lines[bno].decode('utf-8').strip("\n").split(",")
                bbox_4pts = [float(x) for x in line[0:8]]
                left = min(bbox_4pts[::2])
                right = max(bbox_4pts[::2])
                top = min(bbox_4pts[1::2])
                bottom = max(bbox_4pts[1::2])
                bbox = [left, top, right, bottom]
                text = line[8]
                otstr = "%s\t%d %d %d %d\t%d %d\t%s\n" % (
                    text, bbox[0], bbox[1], bbox[2], bbox[3],
                    img_width, img_height, img_path)
                fouti.write(otstr.encode('utf-8'))

                bbox = get_norm_resize_bbox(bbox, img_height, img_width, max_norm_size)
                otstr = "%s\t%d %d %d %d\n" % (
                    text, bbox[0], bbox[1], bbox[2], bbox[3])
                foutb.write(otstr.encode('utf-8')) 
            otstr = "\n"
            foutb.write(otstr.encode('utf-8'))
            fouti.write(otstr.encode('utf-8'))

    foutb.close()
    fouti.close()
    print("ok")
# gen_funds_format()

In [1]:
import os, sys
# from create_pretrain_data import gen_nlpbert_format, read_features_h5py_to_np
# from create_pretrain_data import write_features_np_to_h5py
from create_pretrain_data import merge_h5py
import numpy as np
# gen_nlpbert_format(total_num=12, cur_no=2)

# set_path = "./data/yanbao_v1/"
# input_file = set_path + "yanbao_v1_train.h5.%d" % (0)
# inputs = read_features_h5py_to_np(input_file)
# print(len(inputs))
# for tmp in inputs:
#     print(tmp.shape)

merge_h5py()

grep: warning: GREP_OPTIONS is deprecated; please use an alias or script
/usr/local/lib/python3.7/site-packages/scipy/stats/morestats.py:12: DeprecationWarning: Importing from numpy.testing.decorators is deprecated since numpy 1.15.0, import from numpy.testing instead.
  from numpy.testing.decorators import setastest
[2021-09-06 09:27:20,574] [    INFO] - Already cached ./.paddlenlp/models/ernie-gram-zh/vocab.txt


saving data
5
(161210, 512)
(161210, 1, 512)
(161210, 512, 4)
(161210, 20)
(161210, 20)
ok
